In [1]:
from urllib.request  import urlopen
import bs4
import datetime as dt
import pandas as pd
import re
from pandas import Series, DataFrame
import numpy as np

In [2]:
index_cd='KPI200'
page_n=1
naver_index='https://finance.naver.com/sise/sise_index_day.nhn?code=' +index_cd+'&page='+str(page_n)
from urllib.request import urlopen
source=urlopen(naver_index).read()
import bs4
source=bs4.BeautifulSoup(source,'lxml')
print(source.prettify())

<html lang="ko">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   네이버 금융
  </title>
  <link href="/css/newstock.css?20190411143845" rel="stylesheet" type="text/css"/>
  <link href="/css/common.css?20190411143845" rel="stylesheet" type="text/css"/>
  <link href="/css/layout.css?20190411143845" rel="stylesheet" type="text/css"/>
  <link href="/css/main.css?20190411143845" rel="stylesheet" type="text/css"/>
  <link href="/css/newstock2.css?20190411143845" rel="stylesheet" type="text/css"/>
  <link href="/css/newstock3.css?20190411143845" rel="stylesheet" type="text/css"/>
  <link href="/css/world.css?20190411143845" rel="stylesheet" type="text/css"/>
 </head>
 <body>
  <script src="/js/jindo.min.ns.1.5.3.euckr.js?20190411143845" type="text/javascript">
  </script>
  <script src="/js/lcslog.js?20190411143845" type="text/javascript">
  </script>
  <!-- 일별시세 -->
  <div class="box_type_m">
   <h4 class="top_tlt" style="text-align:left;">
    <em>
   

In [3]:
d=source.find_all('td',class_='date')[0].text
d

'2019.05.24'

In [4]:
import datetime as dt

In [5]:
def date_format(d):
    d = str(d).replace('-','.')
    yyyy = int(d.split('.')[0])
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])
    
    this_date = dt.date(yyyy,mm,dd)
    return this_date

In [6]:
this_close=source.find_all('tr')[2].find_all('td')[1].text
this_close=this_close.replace(',','')
this_close=float(this_close)
this_close

264.42

In [7]:
p=source.find_all('td',class_='number_1')[0].text
p

'264.42'

In [8]:
dates=source.find_all('td',class_='date')
prices=source.find_all('td',class_='number_1')

In [9]:
for n in range(len(dates)):
    this_date = dates[n].text
    this_date = date_format(this_date)
            
    this_close = prices[n*4].text
    this_close = this_close.replace(',','')
    this_close = float(this_close)
    this_close
    
    print(this_date,this_close)

2019-05-24 264.42
2019-05-23 266.38
2019-05-22 266.65
2019-05-21 266.12
2019-05-20 264.96
2019-05-17 264.35


In [10]:
def historical_index_naver(index_cd, start_date='', end_date='', page_n = 1, last_page = 0):
    
    if start_date:
        start_date = date_format(start_date)
    else:
        start_date = dt.date.today()
    if end_date:
        end_date = date_format(end_date)
    else:
        end_date = dt.date.today()
        
    naver_index = 'https://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    source = bs4.BeautifulSoup(urlopen(naver_index).read(), 'lxml')
    dates=source.find_all('td',class_='date')
    prices=source.find_all('td',class_='number_1')
    
    for n in range(len(dates)):
        if dates[n].text.split('.')[0].isdigit():
            this_date = dates[n].text
            this_date = date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:
                this_close = prices[n*4].text
                this_close = this_close.replace(',','')
                this_close = float(this_close)
                historical_prices[this_date] = this_close
            elif this_date < start_date:
                return historical_prices
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        last_page = int(last_page.split('&')[1].split('=')[1])

    if page_n < last_page:
        page_n += 1
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)
        
    return historical_prices


In [11]:
index_cd='KPI200'
historical_prices=dict()
historical_index_naver(index_cd,'2017.03.01','2019.2.28')
historical_prices

{datetime.date(2019, 2, 28): 283.8,
 datetime.date(2019, 2, 27): 288.76,
 datetime.date(2019, 2, 26): 287.89,
 datetime.date(2019, 2, 25): 289.0,
 datetime.date(2019, 2, 22): 288.74,
 datetime.date(2019, 2, 21): 288.41,
 datetime.date(2019, 2, 20): 288.4,
 datetime.date(2019, 2, 19): 284.91,
 datetime.date(2019, 2, 18): 285.9,
 datetime.date(2019, 2, 15): 283.98,
 datetime.date(2019, 2, 14): 288.68,
 datetime.date(2019, 2, 13): 284.97,
 datetime.date(2019, 2, 12): 283.69,
 datetime.date(2019, 2, 11): 282.1,
 datetime.date(2019, 2, 8): 281.49,
 datetime.date(2019, 2, 7): 285.65,
 datetime.date(2019, 2, 1): 285.79,
 datetime.date(2019, 1, 31): 285.89,
 datetime.date(2019, 1, 30): 286.62,
 datetime.date(2019, 1, 29): 283.0,
 datetime.date(2019, 1, 28): 281.64,
 datetime.date(2019, 1, 25): 281.67,
 datetime.date(2019, 1, 24): 276.61,
 datetime.date(2019, 1, 23): 273.87,
 datetime.date(2019, 1, 22): 272.71,
 datetime.date(2019, 1, 21): 274.0,
 datetime.date(2019, 1, 18): 273.88,
 datetime.d

In [12]:
#KOSPI200 일별시세

In [13]:
prices={'kospi200':historical_prices}
kospi200=pd.DataFrame(prices)

In [14]:
future=pd.read_excel("C:/future.xlsx")
future=future[["날짜","현재가"]]
future=future.set_index("날짜")
future=future['2017.03.01':'2019.02.28']

In [15]:
kospi200.sort_index(ascending=True)

,kospi200
2017-03-02,272.65
2017-03-03,269.77
2017-03-06,270.41
2017-03-07,271.87
2017-03-08,272.29
2017-03-09,271.62
2017-03-10,272.29
2017-03-13,275.41
2017-03-14,277.89
2017-03-15,277.89


In [16]:
a=pd.concat([kospi200,future],axis=1)
a

,kospi200,현재가
날짜,,
2017-03-02,272.65,272.7
2017-03-03,269.77,269.6
2017-03-06,270.41,270.2
2017-03-07,271.87,271.6
2017-03-08,272.29,272.2
2017-03-09,271.62,271.6
2017-03-10,272.29,272.6
2017-03-13,275.41,275.8
2017-03-14,277.89,278.0


In [17]:
a.index=pd.to_datetime(a.index)

In [18]:
a["maturity"]='missing'

In [19]:
thur=['2017-03-09','2017-06-08','2017-09-14','2017-12-14','2018-03-08','2018-06-14','2018-09-13','2018-12-13','2019-03-14']
for i in range(len(a)):
    for n in range(0,8):
        if a.index[i].date()<=date_format(thur[0]):
            a["maturity"][i]=thur[0]
        elif a.index[i].date()>date_format(thur[n]) and a.index[i].date()<=date_format(thur[n+1]):
            a["maturity"][i]=thur[n+1]

C:\Users\van\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\van\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [20]:
#36912 둘째주목요일
#이자율 2.25% 배당률 1.50%

def futures_price(S,r,d,T,t0):
    t=np.busday_count(t0.date(),T.date())/252
    F=S*np.exp((r-d)*t)
    return F


In [21]:
thur={'2017-03-09','2017-06-08','2017-09-14','2017-12-14','2018-03-08','2018-06-14','2018-09-13','2018-12-13','2019-03-14'}


r=0.0225
d=0.015
for n in (a.index):
    T=pd.to_datetime(a['maturity'][n])
    t0=n
    S=a.loc[n,'kospi200']
    F=futures_price(S,r,d,T,t0)
    f='{:,.2f}'.format(F)
    a.loc[n,'이론가']=f

In [22]:
a

,kospi200,현재가,maturity,이론가
날짜,,,,
2017-03-02,272.65,272.7,2017-03-09,272.69
2017-03-03,269.77,269.6,2017-03-09,269.80
2017-03-06,270.41,270.2,2017-03-09,270.43
2017-03-07,271.87,271.6,2017-03-09,271.89
2017-03-08,272.29,272.2,2017-03-09,272.30
2017-03-09,271.62,271.6,2017-03-09,271.62
2017-03-10,272.29,272.6,2017-06-08,272.81
2017-03-13,275.41,275.8,2017-06-08,275.93
2017-03-14,277.89,278.0,2017-06-08,278.40


In [23]:
a["basis"]='missing'

In [24]:
for i in range(len(a)):
    a['basis'][i]=round((a['현재가'][i]-a['kospi200'][i]),2)

C:\Users\van\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
a["이론가-현재가"]='missing'
for i in range(len(a)):
    a['이론가-현재가'][i]=round((float(a['이론가'][i])-a['현재가'][i]),2)

C:\Users\van\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
#basis로 계산
margin=0.0
overlap=False
a['margin']=0.0
for i in range(len(a['basis'])):
    if a['basis'][i]>-0.2 and a['basis'][i]<=0.7:
        a['margin'][i]=margin
        continue
    elif a['basis'][i]<=-0.2 and overlap==False:
        margin=margin-a['basis'][i]
        a['margin'][i]=margin
        overlap=True
    elif a['basis'][i]<=-0.2 and overlap==True:
        a['margin'][i]=margin
        continue
    elif a['basis'][i]>0.7 and overlap==True:
        margin=margin+a['basis'][i]
        a['margin'][i]=margin
        overlap=False
    elif a['basis'][i]>0.7 and overlap==False:
        a['margin'][i]=margin
        continue
a['margin'][-1]=a['margin'][-2]+a['basis'][-1]

C:\Users\van\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\van\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\van\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\van\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWa

In [27]:
overlap1=False
a['선물매수현물매도']=0.0
a['선물매도현물매수']=0.0
for i in range(len(a['basis'])):
    if a['basis'][i]>-0.2 and a['basis'][i]<=0.7:
        continue
    elif a['basis'][i]<=-0.2 and overlap1==False:
        a['선물매수현물매도'][i]=1
        overlap1=True
        continue
    elif a['basis'][i]<=-0.2 and overlap1==True:
        continue
    elif a['basis'][i]>0.7 and overlap1==True:
        a['선물매도현물매수'][i]=2
        overlap1=False
        continue
    elif a['basis'][i]>0.7 and overlap1==False:
        continue

C:\Users\van\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\van\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
a

,kospi200,현재가,maturity,이론가,basis,이론가-현재가,margin,선물매수현물매도,선물매도현물매수
날짜,,,,,,,,,
2017-03-02,272.65,272.7,2017-03-09,272.69,0.05,-0.01,0.00,0.0,0.0
2017-03-03,269.77,269.6,2017-03-09,269.80,-0.17,0.2,0.00,0.0,0.0
2017-03-06,270.41,270.2,2017-03-09,270.43,-0.21,0.23,0.21,1.0,0.0
2017-03-07,271.87,271.6,2017-03-09,271.89,-0.27,0.29,0.21,0.0,0.0
2017-03-08,272.29,272.2,2017-03-09,272.30,-0.09,0.1,0.21,0.0,0.0
2017-03-09,271.62,271.6,2017-03-09,271.62,-0.02,0.02,0.21,0.0,0.0
2017-03-10,272.29,272.6,2017-06-08,272.81,0.31,0.21,0.21,0.0,0.0
2017-03-13,275.41,275.8,2017-06-08,275.93,0.39,0.13,0.21,0.0,0.0
2017-03-14,277.89,278.0,2017-06-08,278.40,0.11,0.4,0.21,0.0,0.0


In [29]:
차익거래수익=a['margin'][-1]
차익거래수익*250000

6087499.999999999

In [30]:
a.to_excel("basis.xlsx")